## Part II - Add Ltitude & Longitude to Postal Codes
## Segmenting and Clustering Neighborhoods in Toronto

 ###### Code to Scrape Wikipedia page

In [238]:
# Import Libraries
import pandas as pd
import numpy as np
import requests

In [239]:
# Form URL to fetch the file
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
file = requests.get(url)

In [241]:
# Webpage is extracted into a Text file
file

<Response [200]>

##### Use pandas read_html() command to scrape and parse 'HTML and XML code into Pandas Dataframe

In [242]:
file_df = pd.read_html(file.text)

In [243]:
len(file_df)

3

In [244]:
type(file_df)

list

##### Use the first element of the list which contains required data.

In [245]:
toronto_df = file_df[0]
print(toronto_df.head())
toronto_df.shape


  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


(288, 3)

##### Processing cells with assigned 'Borough'. Filter out 'Not assigned'

In [246]:
toronto_df_filtered =toronto_df[toronto_df['Borough'] != 'Not assigned']

In [247]:
print (toronto_df_filtered[0:10])
toronto_df_filtered.shape

   Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Not assigned
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern


(211, 3)

##### Further replace cells having value as 'Not Assigned' Neighborhood, with the same as the Borough.


In [248]:
toronto_df_neigh_replaced = toronto_df_filtered[:]
cond = toronto_df_neigh_replaced.Neighbourhood == 'Not assigned'
toronto_df_neigh_replaced.loc[cond, 'Neighbourhood'] = toronto_df_neigh_replaced.Borough
print(toronto_df_neigh_replaced[0:10])

   Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Queen's Park
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern


##### Concatenate Neighbourhoods with same Postcodes and Boroughs.
##### Use Groupby method on 'Postcode' and 'Borough' and apply 'function lambda' to concatenate Neighbourhood (separated by ',')

In [249]:
toronto_final_df = toronto_df_neigh_replaced.groupby(["Postcode","Borough"])["Neighbourhood"].apply(lambda x:"%s" % ', '.join(x)).reset_index()
toronto_final_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [251]:
toronto_final_df.shape

(103, 3)

#### Use Google API or CSV file to get latitude and longitude for the latitude and the longitude coordinates of each neighborhood.

In [252]:
url1 = 'https://geocoder.readthedocs.io/index.html'
url2 ='https://cocl.us/Geospatial_data'

In [253]:
ll_file2 = requests.get(url2)
df_geo = pd.read_csv(url2)

In [255]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [256]:
# Use geodecoder to fetch  
import geocoder
for p in df['Postal Code'][:5]:
     g1 = geocoder.google('{}, Toronto'.format(p))
     print(g1.latlng)   

None
None
None
None
None


#### Fetch latitude and longitude coordinates for a postal code from csv file 

In [259]:
# Fetch latitude and longitude coordinates for a postal code from csv file , store them as a list and append lists to the dataframe.
lat =[]
lng=[]

for pcode in toronto_final_df['Postcode']:
    i=0
    for i, p in enumerate(df_geo['Postal Code']):
             if pcode == p: 
                lat.append(df_geo['Latitude'][i])
                lng.append(df_geo['Longitude'][i])
                j=j+1
                #print(pcode, i,p,df_geo['Latitude'][i],df_geo['Longitude'][i])

df_toronto_new= toronto_final_df.assign(Latitude=lat,Longitude=lng)
             
  

Toronto dataframe after appending with coordinates data

In [261]:
df_new.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
